# CROWD ~

**Современная экономика детерминирована желанием и возможностями людей совершать покупки и проводить транзакции, получать услуги. Для этого предпринимателям необходимо как можно более точно знать, куда хотят пойти люди, сколько денег они хотят потратить и в каких услугах они заинтересованы, идеальный торговый центр — это заведение, в котором для каждого есть желаемая покупка, всегда работает эскалатор и всегда есть мыло в туалете.**

**Актуальность нашего проекта очевидна: мы исходим из гипотезы, что, исходя из заданных параметров, можно сделать более-менее точные предположения о предпочтениях людей.**

---

Авторы датасета: t.me/apfel t.me/markus_lons 

---

In [61]:
#Вспомогательное
import pandas as pd # для работы с датасетами
import numpy as np # для работы с листами, математическая обработка
from sklearn.metrics import accuracy_score # точность совпадения
from sklearn import preprocessing # предобработка датасета
from sklearn import metrics # различные метрики
from sklearn.model_selection import train_test_split # для разбиения на обучающую и тестовую выборки
from sklearn.model_selection import cross_val_score # ну всё понятно, умное слово
#Дерево
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#Линейная регрессия
from sklearn.linear_model import LinearRegression # модель линейной
#Логическая регрессия
from sklearn.linear_model import LogisticRegression # модель логический
#Нейронка
from keras.models import Sequential # очень важная штука
from keras.layers import Dense # создает слой нейронов
#Отдельная либа XGBoost
import xgboost as xgb

### Обработка датасета для работы с ним

In [62]:
df = pd.read_csv("temp.csv", sep=";").drop(["Unnamed: 0"], axis=1)

In [63]:
df

,date,year,mounth,day,weekday,temp,pres,prec,wind,absenteeism,...,1,2,3,4,5,6,7,8,9,10
0,20120114,2012,1,14,6,-21,756,0,1.0,4,...,0,0,0,1,0,0,0,0,0,0
1,20120116,2012,1,16,1,-19,758,0,2.0,3,...,0,0,1,0,0,0,0,0,0,0
2,20120117,2012,1,17,2,-27,770,0,1.0,3,...,0,0,1,0,0,0,0,0,0,0
3,20120118,2012,1,18,3,-28,776,0,2.0,6,...,0,0,0,0,0,1,0,0,0,0
4,20120119,2012,1,19,4,-27,778,0,2.0,4,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,20181225,2018,12,25,2,-23,770,0,1.0,2,...,0,1,0,0,0,0,0,0,0,0
1690,20181226,2018,12,26,3,-22,770,0,NaN,1,...,1,0,0,0,0,0,0,0,0,0
1691,20181227,2018,12,27,4,-23,770,0,NaN,1,...,1,0,0,0,0,0,0,0,0,0
1692,20181228,2018,12,28,5,-25,766,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
X = df[["year", "mounth", "day", "weekday", "temp", "pres", "prec", "wind"]]
X = X.apply(pd.to_numeric).fillna(0).astype("int")

In [65]:
X

,year,mounth,day,weekday,temp,pres,prec,wind
0,2012,1,14,6,-21,756,0,1
1,2012,1,16,1,-19,758,0,2
2,2012,1,17,2,-27,770,0,1
3,2012,1,18,3,-28,776,0,2
4,2012,1,19,4,-27,778,0,2
...,...,...,...,...,...,...,...,...
1689,2018,12,25,2,-23,770,0,1
1690,2018,12,26,3,-22,770,0,0
1691,2018,12,27,4,-23,770,0,0
1692,2018,12,28,5,-25,766,0,0


In [66]:
y = df[["absenteeism","0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]].astype("int")

In [67]:
y

,absenteeism,0,1,2,3,4,5,6,7,8,9,10
0,4,0,0,0,0,1,0,0,0,0,0,0
1,3,0,0,0,1,0,0,0,0,0,0,0
2,3,0,0,0,1,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1689,2,0,0,1,0,0,0,0,0,0,0,0
1690,1,0,1,0,0,0,0,0,0,0,0,0
1691,1,0,1,0,0,0,0,0,0,0,0,0
1692,0,1,0,0,0,0,0,0,0,0,0,0


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

Тут мы делим **y** на абсолютное и матричное значение:

In [69]:
y_train_value, y_test_value = y_train["absenteeism"], y_test["absenteeism"]

In [70]:
y_test_matrix, y_train_matrix = y_test.drop(["absenteeism"], axis=1), y_train.drop(["absenteeism"], axis=1)

### Вспомогательные функции

In [71]:
def show_delta(real,predicted,toint=True): # показывает, на сколько отличается реальное от предсказанного.
    if toint:
        delta = [[i[0], int(i[1]), int(i[1]) - i[0]] for i in list(zip(real, predicted))]
    else:
        delta = [[i[0], round(i[1],2), round(i[1] - i[0],2)] for i in list(zip(real, predicted))]
    return delta

# На основе значения

### Дерево

In [72]:
kfold = StratifiedKFold(n_splits=3) # задаем объект для разделения данных на фолды при кросс-валидации

model = DecisionTreeClassifier()

params = {'max_depth': range(1,50)}  # перебираемые параметры модели

grid = GridSearchCV(model, params, cv=kfold, scoring='accuracy', n_jobs=-1)

In [73]:
grid.fit(X_train, y_train_value)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
       

In [74]:
model_tree = grid.best_estimator_
model_tree

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [75]:
y_pred_tree = model_tree.predict(X_test)

In [76]:
print('\nScore test: \n', model_tree.score(X_test, y_test_value))
print('\nScore train: \n', model_tree.score(X_train, y_train_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_tree))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_tree), "\n")
print(*show_delta(y_test_value, y_pred_tree), sep="\n")


Score test: 
 0.3614931237721022

Score train: 
 0.7561181434599156

Mean absolute error: 
 0.9705304518664047

Median absolute error: 
 1.0 

[4, 3, -1]
[2, 2, 0]
[3, 4, 1]
[4, 6, 2]
[3, 2, -1]
[5, 3, -2]
[5, 2, -3]
[0, 0, 0]
[2, 2, 0]
[3, 3, 0]
[7, 3, -4]
[0, 0, 0]
[3, 2, -1]
[2, 2, 0]
[3, 2, -1]
[5, 4, -1]
[1, 4, 3]
[3, 2, -1]
[5, 7, 2]
[2, 2, 0]
[2, 3, 1]
[3, 2, -1]
[0, 2, 2]
[2, 4, 2]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[3, 4, 1]
[3, 4, 1]
[3, 4, 1]
[5, 3, -2]
[7, 5, -2]
[1, 3, 2]
[0, 1, 1]
[4, 3, -1]
[1, 1, 0]
[2, 2, 0]
[3, 2, -1]
[2, 5, 3]
[6, 7, 1]
[3, 3, 0]
[1, 4, 3]
[2, 1, -1]
[0, 1, 1]
[3, 3, 0]
[2, 3, 1]
[2, 2, 0]
[0, 0, 0]
[6, 4, -2]
[2, 2, 0]
[3, 4, 1]
[3, 3, 0]
[4, 1, -3]
[2, 2, 0]
[0, 1, 1]
[2, 2, 0]
[6, 1, -5]
[2, 3, 1]
[4, 4, 0]
[3, 3, 0]
[2, 3, 1]
[3, 3, 0]
[3, 3, 0]
[1, 0, -1]
[6, 3, -3]
[6, 5, -1]
[4, 4, 0]
[2, 4, 2]
[3, 3, 0]
[4, 4, 0]
[2, 3, 1]
[3, 3, 0]
[5, 3, -2]
[1, 2, 1]
[2, 2, 0]
[3, 5, 2]
[1, 3, 2]
[1, 0, -1]
[1, 0, -1]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[2, 2, 0]
[4

### XGBoost

In [77]:
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train, y_train_value)
y_pred_xgb = model_xgb.predict(X_test)

In [78]:
print('\nScore test: \n', accuracy_score(list(y_test["absenteeism"]), y_pred_xgb))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_xgb))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_xgb), "\n")
print(*show_delta(y_test_value, y_pred_xgb), sep="\n")


Score test: 
 0.412573673870334

Mean absolute error: 
 0.899803536345776

Median absolute error: 
 1.0 

[4, 3, -1]
[2, 2, 0]
[3, 3, 0]
[4, 4, 0]
[3, 2, -1]
[5, 4, -1]
[5, 1, -4]
[0, 0, 0]
[2, 5, 3]
[3, 4, 1]
[7, 3, -4]
[0, 0, 0]
[3, 2, -1]
[2, 2, 0]
[3, 2, -1]
[5, 3, -2]
[1, 4, 3]
[3, 2, -1]
[5, 8, 3]
[2, 2, 0]
[2, 3, 1]
[3, 3, 0]
[0, 0, 0]
[2, 4, 2]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[3, 2, -1]
[3, 5, 2]
[3, 3, 0]
[5, 4, -1]
[7, 5, -2]
[1, 3, 2]
[0, 1, 1]
[4, 3, -1]
[1, 1, 0]
[2, 1, -1]
[3, 3, 0]
[2, 3, 1]
[6, 8, 2]
[3, 3, 0]
[1, 0, -1]
[2, 3, 1]
[0, 1, 1]
[3, 3, 0]
[2, 3, 1]
[2, 2, 0]
[0, 0, 0]
[6, 4, -2]
[2, 3, 1]
[3, 4, 1]
[3, 3, 0]
[4, 4, 0]
[2, 1, -1]
[0, 1, 1]
[2, 2, 0]
[6, 2, -4]
[2, 3, 1]
[4, 3, -1]
[3, 3, 0]
[2, 3, 1]
[3, 3, 0]
[3, 3, 0]
[1, 1, 0]
[6, 2, -4]
[6, 3, -3]
[4, 2, -2]
[2, 2, 0]
[3, 2, -1]
[4, 4, 0]
[2, 2, 0]
[3, 2, -1]
[5, 2, -3]
[1, 1, 0]
[2, 3, 1]
[3, 3, 0]
[1, 2, 1]
[1, 1, 0]
[1, 2, 1]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[2, 2, 0]
[4, 5, 1]
[2, 3, 1]
[3, 4, 1]
[2, 3, 1

### Линейная регрессия

In [79]:
LR = LinearRegression()
model_lin = LR.fit(X_train, y_train_value)
y_pred_lin = model_lin.predict(X_test)

In [80]:
linear_scoring = cross_val_score(model_lin, X_train, y_train_value, scoring="neg_mean_absolute_error", cv=10)
print(f"{linear_scoring.mean()}, {linear_scoring.std()}")

-1.0438388851169351, 0.09302366642345365


In [81]:
print('\nIntercept: \n', model_lin.intercept_)
print('\nScore test: \n', model_lin.score(X_test, y_test_value))
print('\nScore train: \n', model_lin.score(X_train, y_train_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_lin))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_lin), "\n")
print(*show_delta(y_test_value, y_pred_lin, toint=False), sep="\n")


Intercept: 
 -576.0353682201463

Score test: 
 0.3690090068680405

Score train: 
 0.3795763702447287

Mean absolute error: 
 1.0452527150972954

Median absolute error: 
 0.819731175196921 

[4, 3.17, -0.83]
[2, 0.55, -1.45]
[3, 3.19, 0.19]
[4, 4.99, 0.99]
[3, 2.28, -0.72]
[5, 4.83, -0.17]
[5, 2.78, -2.22]
[0, 2.46, 2.46]
[2, 2.34, 0.34]
[3, 3.9, 0.9]
[7, 3.96, -3.04]
[0, 1.04, 1.04]
[3, 2.41, -0.59]
[2, 2.17, 0.17]
[3, 2.33, -0.67]
[5, 2.68, -2.32]
[1, 2.72, 1.72]
[3, 2.51, -0.49]
[5, 3.2, -1.8]
[2, 2.0, -0.0]
[2, 3.96, 1.96]
[3, 1.74, -1.26]
[0, 1.73, 1.73]
[2, 4.46, 2.46]
[2, 2.38, 0.38]
[3, 2.72, -0.28]
[3, 2.94, -0.06]
[3, 3.32, 0.32]
[3, 4.2, 1.2]
[3, 4.11, 1.11]
[5, 4.35, -0.65]
[7, 5.2, -1.8]
[1, 3.02, 2.02]
[0, 1.98, 1.98]
[4, 3.38, -0.62]
[1, 2.75, 1.75]
[2, 1.76, -0.24]
[3, 2.04, -0.96]
[2, 3.31, 1.31]
[6, 3.82, -2.18]
[3, 3.9, 0.9]
[1, 2.46, 1.46]
[2, 2.35, 0.35]
[0, 1.17, 1.17]
[3, 4.25, 1.25]
[2, 2.78, 0.78]
[2, 2.55, 0.55]
[0, 1.3, 1.3]
[6, 4.28, -1.72]
[2, 2.52, 0.52]
[

In [82]:
model_lin.predict([[2018, 12, 6, 5, -21, 770, 0, 1]])

array([4.7458333])

---

# Матричные

### Нейронка

In [83]:
def nn_num(X_train_local, y_train_local, epochs=1000, batch=12):
    model = Sequential()
    model.add(Dense(12, input_dim=len(list(X_train_local)), activation='relu'))
    model.add(Dense(122, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    model.fit(X_train_local, y_train_local, epochs = epochs, batch_size=batch)
    return model

In [84]:
models_nn = [None]*len(list(y_train_matrix))
for i in range(len(models_nn)):
    models_nn[i] = nn_num(X_train, y_train_matrix[str(i)], epochs=1000, batch=20)

Epoch 1/1000
1185/1185 [==============================] - 0s 192us/step - loss: 1.9041 - accuracy: 0.8473
Epoch 2/1000
1185/1185 [==============================] - 0s 73us/step - loss: 0.6795 - accuracy: 0.8759
Epoch 3/1000
1185/1185 [==============================] - 0s 79us/step - loss: 0.7882 - accuracy: 0.8574
Epoch 4/1000
1185/1185 [==============================] - 0s 70us/step - loss: 0.7309 - accuracy: 0.8616
Epoch 5/1000
1185/1185 [==============================] - 0s 61us/step - loss: 0.4572 - accuracy: 0.8903
Epoch 6/1000
1185/1185 [==============================] - 0s 58us/step - loss: 0.5659 - accuracy: 0.8405
Epoch 7/1000
1185/1185 [==============================] - 0s 54us/step - loss: 0.3820 - accuracy: 0.9013
Epoch 8/1000
1185/1185 [==============================] - 0s 62us/step - loss: 0.4342 - accuracy: 0.8920
Epoch 9/1000
1185/1185 [==============================] - 0s 50us/step - loss: 0.6136 - accuracy: 0.8557
Epoch 10/1000
1185/1185 [=============================

1185/1185 [==============================] - 0s 48us/step - loss: 0.2584 - accuracy: 0.9030
Epoch 155/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2593 - accuracy: 0.9089
Epoch 156/1000
1185/1185 [==============================] - 0s 57us/step - loss: 0.2549 - accuracy: 0.9097
Epoch 157/1000
1185/1185 [==============================] - 0s 56us/step - loss: 0.2567 - accuracy: 0.9063
Epoch 158/1000
1185/1185 [==============================] - 0s 74us/step - loss: 0.2574 - accuracy: 0.9063
Epoch 159/1000
1185/1185 [==============================] - 0s 54us/step - loss: 0.2697 - accuracy: 0.9072
Epoch 160/1000
1185/1185 [==============================] - 0s 60us/step - loss: 0.2575 - accuracy: 0.9072
Epoch 161/1000
1185/1185 [==============================] - 0s 94us/step - loss: 0.2563 - accuracy: 0.9072
Epoch 162/1000
1185/1185 [==============================] - 0s 54us/step - loss: 0.2581 - accuracy: 0.9021
Epoch 163/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 50us/step - loss: 0.2633 - accuracy: 0.9038
Epoch 307/1000
1185/1185 [==============================] - 0s 47us/step - loss: 0.2517 - accuracy: 0.9072
Epoch 308/1000
1185/1185 [==============================] - 0s 53us/step - loss: 0.2611 - accuracy: 0.9072
Epoch 309/1000
1185/1185 [==============================] - 0s 53us/step - loss: 0.2640 - accuracy: 0.9072
Epoch 310/1000
1185/1185 [==============================] - 0s 42us/step - loss: 0.2509 - accuracy: 0.9072
Epoch 311/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2478 - accuracy: 0.9072
Epoch 312/1000
1185/1185 [==============================] - 0s 40us/step - loss: 0.2546 - accuracy: 0.9072
Epoch 313/1000
1185/1185 [==============================] - 0s 43us/step - loss: 0.2549 - accuracy: 0.9072
Epoch 314/1000
1185/1185 [==============================] - 0s 53us/step - loss: 0.2592 - accuracy: 0.9046
Epoch 315/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 43us/step - loss: 0.2471 - accuracy: 0.9080
Epoch 459/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2500 - accuracy: 0.9072
Epoch 460/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.2522 - accuracy: 0.9046
Epoch 461/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2524 - accuracy: 0.9072
Epoch 462/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2608 - accuracy: 0.9072
Epoch 463/1000
1185/1185 [==============================] - 0s 56us/step - loss: 0.2495 - accuracy: 0.9072
Epoch 464/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2506 - accuracy: 0.9072
Epoch 465/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2625 - accuracy: 0.9063
Epoch 466/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2504 - accuracy: 0.9072
Epoch 467/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 47us/step - loss: 0.2635 - accuracy: 0.9072
Epoch 611/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2619 - accuracy: 0.9072
Epoch 612/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.2476 - accuracy: 0.9072
Epoch 613/1000
1185/1185 [==============================] - 0s 50us/step - loss: 0.2607 - accuracy: 0.9063
Epoch 614/1000
1185/1185 [==============================] - 0s 58us/step - loss: 0.2550 - accuracy: 0.9063
Epoch 615/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2479 - accuracy: 0.9055
Epoch 616/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.2438 - accuracy: 0.9089
Epoch 617/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.2482 - accuracy: 0.9072
Epoch 618/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.2496 - accuracy: 0.9089
Epoch 619/1000
1185/1185 [==========================

...

1185/1185 [==============================] - 0s 49us/step - loss: 0.0091 - accuracy: 0.9983
Epoch 356/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0072 - accuracy: 0.9983
Epoch 357/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.0082 - accuracy: 0.9975
Epoch 358/1000
1185/1185 [==============================] - 0s 46us/step - loss: 0.0058 - accuracy: 0.9975
Epoch 359/1000
1185/1185 [==============================] - 0s 43us/step - loss: 0.0064 - accuracy: 0.9983
Epoch 360/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0066 - accuracy: 0.9983
Epoch 361/1000
1185/1185 [==============================] - 0s 53us/step - loss: 0.0071 - accuracy: 0.9975
Epoch 362/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.0073 - accuracy: 0.9983
Epoch 363/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0071 - accuracy: 0.9992
Epoch 364/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 49us/step - loss: 0.0065 - accuracy: 0.9983
Epoch 508/1000
1185/1185 [==============================] - 0s 55us/step - loss: 0.0079 - accuracy: 0.9983
Epoch 509/1000
1185/1185 [==============================] - 0s 50us/step - loss: 0.0068 - accuracy: 0.9983
Epoch 510/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0065 - accuracy: 0.9983
Epoch 511/1000
1185/1185 [==============================] - 0s 47us/step - loss: 0.0075 - accuracy: 0.9983
Epoch 512/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0075 - accuracy: 0.9966
Epoch 513/1000
1185/1185 [==============================] - 0s 47us/step - loss: 0.0082 - accuracy: 0.9975
Epoch 514/1000
1185/1185 [==============================] - 0s 46us/step - loss: 0.0065 - accuracy: 0.9983
Epoch 515/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.0068 - accuracy: 0.9975
Epoch 516/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 50us/step - loss: 0.0078 - accuracy: 0.9975
Epoch 660/1000
1185/1185 [==============================] - 0s 60us/step - loss: 0.0073 - accuracy: 0.9992
Epoch 661/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0068 - accuracy: 0.9975
Epoch 662/1000
1185/1185 [==============================] - 0s 44us/step - loss: 0.0060 - accuracy: 0.9983
Epoch 663/1000
1185/1185 [==============================] - 0s 45us/step - loss: 0.0064 - accuracy: 0.9975
Epoch 664/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0057 - accuracy: 0.9983
Epoch 665/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0072 - accuracy: 0.9975
Epoch 666/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.0063 - accuracy: 0.9975
Epoch 667/1000
1185/1185 [==============================] - 0s 52us/step - loss: 0.0056 - accuracy: 0.9983
Epoch 668/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 47us/step - loss: 0.0066 - accuracy: 0.9983
Epoch 812/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0058 - accuracy: 0.9983
Epoch 813/1000
1185/1185 [==============================] - 0s 46us/step - loss: 0.0057 - accuracy: 0.9975
Epoch 814/1000
1185/1185 [==============================] - 0s 50us/step - loss: 0.0063 - accuracy: 0.9983
Epoch 815/1000
1185/1185 [==============================] - 0s 47us/step - loss: 0.0063 - accuracy: 0.9983
Epoch 816/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0061 - accuracy: 0.9975
Epoch 817/1000
1185/1185 [==============================] - 0s 41us/step - loss: 0.0062 - accuracy: 0.9958
Epoch 818/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.0060 - accuracy: 0.9983
Epoch 819/1000
1185/1185 [==============================] - 0s 44us/step - loss: 0.0066 - accuracy: 0.9983
Epoch 820/1000
1185/1185 [==========================

1185/1185 [==============================] - 0s 55us/step - loss: 0.0060 - accuracy: 0.9983
Epoch 964/1000
1185/1185 [==============================] - 0s 47us/step - loss: 0.0064 - accuracy: 0.9975
Epoch 965/1000
1185/1185 [==============================] - 0s 51us/step - loss: 0.0060 - accuracy: 0.9983
Epoch 966/1000
1185/1185 [==============================] - 0s 53us/step - loss: 0.0053 - accuracy: 0.9992
Epoch 967/1000
1185/1185 [==============================] - 0s 46us/step - loss: 0.0071 - accuracy: 0.9983
Epoch 968/1000
1185/1185 [==============================] - 0s 46us/step - loss: 0.0063 - accuracy: 0.9975
Epoch 969/1000
1185/1185 [==============================] - 0s 60us/step - loss: 0.0068 - accuracy: 0.9983
Epoch 970/1000
1185/1185 [==============================] - 0s 48us/step - loss: 0.0058 - accuracy: 0.9992
Epoch 971/1000
1185/1185 [==============================] - 0s 49us/step - loss: 0.0072 - accuracy: 0.9983
Epoch 972/1000
1185/1185 [==========================

In [85]:
predicted_nn = [None]*len(models_nn)
for model in enumerate(models_nn):
    predicted_nn[model[0]] = model[1].predict(X_test, batch_size=10)
y_pred_nn = [[round(float(j[i][0]),2) for j in list(predicted_nn)] for i in range(len(predicted_nn[0]))]

In [86]:
y_pred_nn

[[0.0, 0.02, 0.28, 0.28, 0.19, 0.06, 0.09, 0.03, 0.0, 0.0, 0.0],
 [0.18, 0.19, 0.21, 0.28, 0.04, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.02, 0.05, 0.22, 0.28, 0.14, 0.13, 0.02, 0.01, 0.01, 0.0, 0.0],
 [0.0, 0.01, 0.1, 0.28, 0.21, 0.18, 0.09, 0.08, 0.0, 0.01, 0.0],
 [0.17, 0.19, 0.21, 0.28, 0.12, 0.08, 0.01, 0.0, 0.0, 0.0, 0.0],
 [0.01, 0.02, 0.16, 0.28, 0.2, 0.18, 0.06, 0.02, 0.01, 0.0, 0.0],
 [0.41, 0.19, 0.1, 0.28, 0.03, 0.05, 0.01, 0.0, 0.0, 0.0, 0.0],
 [0.17, 0.14, 0.28, 0.28, 0.15, 0.07, 0.02, 0.0, 0.0, 0.0, 0.0],
 [0.03, 0.11, 0.22, 0.28, 0.21, 0.18, 0.04, 0.02, 0.0, 0.0, 0.0],
 [0.02, 0.04, 0.19, 0.28, 0.2, 0.06, 0.04, 0.01, 0.02, 0.0, 0.0],
 [0.06, 0.11, 0.14, 0.28, 0.21, 0.08, 0.03, 0.01, 0.01, 0.0, 0.0],
 [0.13, 0.16, 0.18, 0.28, 0.04, 0.02, 0.01, 0.0, 0.0, 0.0, 0.0],
 [0.03, 0.06, 0.28, 0.28, 0.17, 0.04, 0.03, 0.03, 0.0, 0.0, 0.0],
 [0.04, 0.13, 0.19, 0.28, 0.14, 0.13, 0.01, 0.0, 0.0, 0.0, 0.0],
 [0.02, 0.1, 0.21, 0.28, 0.18, 0.09, 0.03, 0.0, 0.0, 0.0, 0.0],
 [0.02, 0.13, 0.28, 

In [87]:
y_pred_nn_value = [i.index(max(i)) for i in y_pred_nn]

In [88]:
show_delta(list(y_test["absenteeism"]), y_pred_nn_value)

[[4, 2, -2],
 [2, 3, 1],
 [3, 3, 0],
 [4, 3, -1],
 [3, 3, 0],
 [5, 3, -2],
 [5, 0, -5],
 [0, 2, 2],
 [2, 3, 1],
 [3, 3, 0],
 [7, 3, -4],
 [0, 3, 3],
 [3, 2, -1],
 [2, 3, 1],
 [3, 3, 0],
 [5, 2, -3],
 [1, 3, 2],
 [3, 2, -1],
 [5, 3, -2],
 [2, 3, 1],
 [2, 3, 1],
 [3, 3, 0],
 [0, 0, 0],
 [2, 2, 0],
 [2, 3, 1],
 [3, 3, 0],
 [3, 2, -1],
 [3, 3, 0],
 [3, 3, 0],
 [3, 3, 0],
 [5, 2, -3],
 [7, 3, -4],
 [1, 3, 2],
 [0, 3, 3],
 [4, 3, -1],
 [1, 0, -1],
 [2, 3, 1],
 [3, 3, 0],
 [2, 3, 1],
 [6, 3, -3],
 [3, 3, 0],
 [1, 0, -1],
 [2, 2, 0],
 [0, 2, 2],
 [3, 3, 0],
 [2, 3, 1],
 [2, 3, 1],
 [0, 2, 2],
 [6, 3, -3],
 [2, 2, 0],
 [3, 3, 0],
 [3, 3, 0],
 [4, 3, -1],
 [2, 3, 1],
 [0, 0, 0],
 [2, 2, 0],
 [6, 3, -3],
 [2, 0, -2],
 [4, 3, -1],
 [3, 3, 0],
 [2, 2, 0],
 [3, 2, -1],
 [3, 2, -1],
 [1, 3, 2],
 [6, 3, -3],
 [6, 3, -3],
 [4, 2, -2],
 [2, 2, 0],
 [3, 3, 0],
 [4, 3, -1],
 [2, 3, 1],
 [3, 3, 0],
 [5, 2, -3],
 [1, 2, 1],
 [2, 2, 0],
 [3, 3, 0],
 [1, 0, -1],
 [1, 0, -1],
 [1, 3, 2],
 [3, 3, 0],
 [3, 3, 0]

In [89]:
print('\nScore test: \n', accuracy_score(list(y_test["absenteeism"]), [i.index(max(i)) for i in y_pred_nn]))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_nn_value))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_nn_value), "\n")
print(*show_delta(y_test_value, y_pred_nn_value), sep="\n")


Score test: 
 0.3005893909626719

Mean absolute error: 
 1.2455795677799608

Median absolute error: 
 1.0 

[4, 2, -2]
[2, 3, 1]
[3, 3, 0]
[4, 3, -1]
[3, 3, 0]
[5, 3, -2]
[5, 0, -5]
[0, 2, 2]
[2, 3, 1]
[3, 3, 0]
[7, 3, -4]
[0, 3, 3]
[3, 2, -1]
[2, 3, 1]
[3, 3, 0]
[5, 2, -3]
[1, 3, 2]
[3, 2, -1]
[5, 3, -2]
[2, 3, 1]
[2, 3, 1]
[3, 3, 0]
[0, 0, 0]
[2, 2, 0]
[2, 3, 1]
[3, 3, 0]
[3, 2, -1]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[5, 2, -3]
[7, 3, -4]
[1, 3, 2]
[0, 3, 3]
[4, 3, -1]
[1, 0, -1]
[2, 3, 1]
[3, 3, 0]
[2, 3, 1]
[6, 3, -3]
[3, 3, 0]
[1, 0, -1]
[2, 2, 0]
[0, 2, 2]
[3, 3, 0]
[2, 3, 1]
[2, 3, 1]
[0, 2, 2]
[6, 3, -3]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[4, 3, -1]
[2, 3, 1]
[0, 0, 0]
[2, 2, 0]
[6, 3, -3]
[2, 0, -2]
[4, 3, -1]
[3, 3, 0]
[2, 2, 0]
[3, 2, -1]
[3, 2, -1]
[1, 3, 2]
[6, 3, -3]
[6, 3, -3]
[4, 2, -2]
[2, 2, 0]
[3, 3, 0]
[4, 3, -1]
[2, 3, 1]
[3, 3, 0]
[5, 2, -3]
[1, 2, 1]
[2, 2, 0]
[3, 3, 0]
[1, 0, -1]
[1, 0, -1]
[1, 3, 2]
[3, 3, 0]
[3, 3, 0]
[3, 2, -1]
[2, 2, 0]
[4, 3, -1]
[2, 3, 1]
[3, 2, -1

### Логическая регрессия. Почти ничем не отличается от нейронки

In [90]:
def logreg_num(X_train_local, y_train_local):
    LR = LinearRegression()
    model = LR.fit(X_train_local, y_train_local)
    return model

In [91]:
models_logreg = [None]*len(list(y_train_matrix))
for i in range(len(models_logreg)):
    models_logreg[i] = logreg_num(X_train, y_train_matrix[str(i)])

In [92]:
predicted_logreg = [None]*len(models_logreg)
for model in enumerate(models_logreg):
    predicted_logreg[model[0]] = model[1].predict(X_test)
y_pred_logreg = [[round(float(j[i]),2) for j in list(predicted_logreg)] for i in range(len(predicted_logreg[0]))]

In [93]:
y_pred_logreg

[[0.05, 0.08, 0.29, 0.31, 0.1, 0.02, 0.07, 0.03, 0.01, 0.02, 0.02],
 [0.29, 0.29, 0.4, 0.2, -0.0, -0.06, -0.04, -0.04, -0.01, -0.02, -0.01],
 [0.03, 0.08, 0.26, 0.25, 0.19, 0.12, 0.04, 0.02, 0.02, -0.0, -0.0],
 [-0.11, -0.02, 0.04, 0.34, 0.32, 0.19, 0.12, 0.06, 0.02, 0.03, 0.01],
 [0.17, 0.2, 0.08, 0.3, 0.16, 0.09, 0.01, 0.0, -0.0, -0.0, -0.01],
 [-0.09, -0.01, 0.01, 0.36, 0.34, 0.2, 0.09, 0.05, 0.03, 0.02, 0.01],
 [0.15, 0.15, 0.01, 0.32, 0.23, 0.11, 0.03, 0.01, 0.01, -0.01, -0.01],
 [0.13, 0.13, 0.28, 0.26, 0.12, 0.03, 0.02, 0.02, -0.01, 0.01, 0.0],
 [0.24, 0.15, 0.21, 0.14, 0.11, 0.06, 0.03, 0.03, 0.01, 0.01, 0.0],
 [-0.01, 0.02, 0.13, 0.35, 0.25, 0.1, 0.07, 0.04, 0.02, 0.01, 0.01],
 [0.03, 0.04, 0.01, 0.37, 0.27, 0.13, 0.07, 0.04, 0.02, 0.01, 0.01],
 [0.24, 0.24, 0.4, 0.22, 0.02, -0.05, -0.02, -0.04, -0.0, -0.02, -0.0],
 [0.17, 0.1, 0.35, 0.2, 0.09, 0.01, 0.03, 0.03, 0.01, 0.01, 0.01],
 [0.13, 0.19, 0.28, 0.21, 0.12, 0.08, 0.01, -0.01, 0.0, -0.01, -0.01],
 [0.12, 0.17, 0.28, 0.23, 

In [94]:
show_delta(list(y_test["absenteeism"]),[i.index(max(i)) for i in y_pred_logreg])

[[4, 3, -1],
 [2, 2, 0],
 [3, 2, -1],
 [4, 3, -1],
 [3, 3, 0],
 [5, 3, -2],
 [5, 3, -2],
 [0, 2, 2],
 [2, 0, -2],
 [3, 3, 0],
 [7, 3, -4],
 [0, 2, 2],
 [3, 2, -1],
 [2, 2, 0],
 [3, 2, -1],
 [5, 3, -2],
 [1, 3, 2],
 [3, 2, -1],
 [5, 2, -3],
 [2, 2, 0],
 [2, 3, 1],
 [3, 0, -3],
 [0, 0, 0],
 [2, 3, 1],
 [2, 2, 0],
 [3, 3, 0],
 [3, 3, 0],
 [3, 3, 0],
 [3, 3, 0],
 [3, 3, 0],
 [5, 3, -2],
 [7, 4, -3],
 [1, 3, 2],
 [0, 2, 2],
 [4, 3, -1],
 [1, 3, 2],
 [2, 2, 0],
 [3, 2, -1],
 [2, 3, 1],
 [6, 3, -3],
 [3, 3, 0],
 [1, 3, 2],
 [2, 2, 0],
 [0, 2, 2],
 [3, 3, 0],
 [2, 3, 1],
 [2, 2, 0],
 [0, 2, 2],
 [6, 3, -3],
 [2, 3, 1],
 [3, 2, -1],
 [3, 2, -1],
 [4, 3, -1],
 [2, 2, 0],
 [0, 3, 3],
 [2, 2, 0],
 [6, 0, -6],
 [2, 2, 0],
 [4, 3, -1],
 [3, 3, 0],
 [2, 2, 0],
 [3, 3, 0],
 [3, 3, 0],
 [1, 3, 2],
 [6, 2, -4],
 [6, 3, -3],
 [4, 3, -1],
 [2, 3, 1],
 [3, 3, 0],
 [4, 3, -1],
 [2, 3, 1],
 [3, 2, -1],
 [5, 2, -3],
 [1, 3, 2],
 [2, 3, 1],
 [3, 3, 0],
 [1, 2, 1],
 [1, 0, -1],
 [1, 3, 2],
 [3, 3, 0],
 [3, 4, 1

In [95]:
y_pred_logreg_value = [i.index(max(i)) for i in y_pred_logreg]
print('\nScore test: \n', accuracy_score(list(y_test["absenteeism"]), y_pred_logreg_value))
print('\nMean absolute error: \n', metrics.mean_absolute_error(y_test_value, y_pred_logreg_value))
print('\nMedian absolute error: \n', metrics.median_absolute_error(y_test_value, y_pred_logreg_value), "\n")
print(*show_delta(y_test_value, y_pred_logreg_value), sep="\n")


Score test: 
 0.33398821218074654

Mean absolute error: 
 1.137524557956778

Median absolute error: 
 1.0 

[4, 3, -1]
[2, 2, 0]
[3, 2, -1]
[4, 3, -1]
[3, 3, 0]
[5, 3, -2]
[5, 3, -2]
[0, 2, 2]
[2, 0, -2]
[3, 3, 0]
[7, 3, -4]
[0, 2, 2]
[3, 2, -1]
[2, 2, 0]
[3, 2, -1]
[5, 3, -2]
[1, 3, 2]
[3, 2, -1]
[5, 2, -3]
[2, 2, 0]
[2, 3, 1]
[3, 0, -3]
[0, 0, 0]
[2, 3, 1]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[3, 3, 0]
[5, 3, -2]
[7, 4, -3]
[1, 3, 2]
[0, 2, 2]
[4, 3, -1]
[1, 3, 2]
[2, 2, 0]
[3, 2, -1]
[2, 3, 1]
[6, 3, -3]
[3, 3, 0]
[1, 3, 2]
[2, 2, 0]
[0, 2, 2]
[3, 3, 0]
[2, 3, 1]
[2, 2, 0]
[0, 2, 2]
[6, 3, -3]
[2, 3, 1]
[3, 2, -1]
[3, 2, -1]
[4, 3, -1]
[2, 2, 0]
[0, 3, 3]
[2, 2, 0]
[6, 0, -6]
[2, 2, 0]
[4, 3, -1]
[3, 3, 0]
[2, 2, 0]
[3, 3, 0]
[3, 3, 0]
[1, 3, 2]
[6, 2, -4]
[6, 3, -3]
[4, 3, -1]
[2, 3, 1]
[3, 3, 0]
[4, 3, -1]
[2, 3, 1]
[3, 2, -1]
[5, 2, -3]
[1, 3, 2]
[2, 3, 1]
[3, 3, 0]
[1, 2, 1]
[1, 0, -1]
[1, 3, 2]
[3, 3, 0]
[3, 4, 1]
[3, 2, -1]
[2, 2, 0]
[4, 3, -1]
[2, 3, 1]
[3, 3, 0

---

# Что теперь имеем?

1. y_pred_tree - model_tree - дерево
2. y_pred_lin - model_lin - линейная
3. y_pred_xgb - model_xgb
4. y_pred_nn - y_pred_nn_value - models_nn (лист с моделями) - нейронка
5. y_pred_logreg - y_pred_logreg_value - models_logreg (лист с моделями) - логическая